In [1]:
def read_chem_master(string):
  # Looks like:
  '''
  ratj_t1(11,'NO2       ','PHOTON    ','NO        ','O(3P)     ','          ',   &
       '          ', 0.0,0.0,0.0,0.0, 100.0,'jno2      ',ti+s+t+st+r+cs,0,0,107),&
  '''
  digits = string.index(',') - 8
  item_num = int(string[8:8+digits])
  reactant = string[9+digits:21+digits] # 10 char spaces
  products =  string[35+digits:74+digits]+string[86:98] # "'NO        ','O(3P)     ','          ',   &\n     '          '" # 4 x 10 char spaces
  rate_label = string[125:137] # "'jno2      '" # 10 char spaces
  schemes = string[138:string.index(',', 138)] # "ti+s+t+st+r+cs"

  return item_num, reactant, products, rate_label, schemes

def reaction(num_chems, reactant, products):
  products_string = products[1:products.index(' ')]
  for i in range(num_chems - 3):
    index = 13*(i+1)+1   # 0 -> 14, 1 -> 27, 2 -> 40
    products_string += " + " + products[index:products.index(' ',index)]
  reaction = "{reac} -> {prods}" \
               .format(reac = reactant[1:reactant.index(' ')], \
                           prods = products_string)
  return reaction

def write_flux_dat(item_num, num_chems, reactant, products):
  # write the asad_flux_dat entry
  # reactant has 10 char spaces
  # products have 4 x 10 char spaces
  # Should look like:       note that spacing might not look consistent when it actually is, depending on text editor.
  '''
  asad_flux_defn('RTE',50500,'J',.FALSE.,0,5,                             &
  ['MeONO2    ','PHOTON    '],                                            &
  ['HO2       ','HCHO      ','NO2       ','          ']),                 &
  '''
  entry = "  asad_flux_defn('RTE',50{item},'J',.FALSE.,0,{chem},                                  &\n  [{reac},'PHOTON    '],                                                 &\n  [{prods}]),                      &" \
              .format(item = item_num, chem = num_chems, reac = reactant, prods = products)
  print(entry)

def write_stashmaster_A(item_num, num_chems, reactant, products):
# Convert chem_master entries into stashmaster entries.
  # name has 36 char spaces.
  reaction_string = reaction(num_chems, reactant, products)
  name = "{reac} J-values" \
               .format(reac = reaction_string)
  if len(name) > 36:
    name = "{reac} J-values" \
                .format(reac = reactant[1:reactant.index(' ')]) 
  while len(name) < 36:
    name = name + ' '
 
  entry = '''1|    1 |   50 |  {item} |{title}|
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000000000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#'''.format(item = item_num, title = name)
  print(entry)

def write_stashmaster_meta(item_num, num_chems, reactant, products):
  reaction_string = reaction(num_chems, reactant, products)
  entry = '''[stashmaster:code(50{item})]
description=Photon + {reac} photolysis rate coefficients.
help=Ideally, use CRI-Strat versions >= 121. Photolysis rate coefficients aka J-rates.
'''.format(item = item_num, reac = reaction_string)
  print(entry)

def write_note(item_num, num_chems, reactant, product):
  print(item_num, ' ', reactant[1:reactant.index(' ')])

def write_from_master(master_string, write_fn, start_item_num):
  # write_fn needs to be one of the write functions.
  num_strings = master_string.count('ratj_t1')
  start_pos = 0
  for i in range(num_strings):
      start = master_string.index('ratj_t1', start_pos) 
      end =  master_string.index('&', start + 141)
      entry = master_string[start : end+1]
      master_item_num, reactant, products, rate_label, schemes = read_chem_master(entry)
      flux_item_num = i + start_item_num
      num_chems = 2 + (4 - products.count('          '))
      write_fn(flux_item_num, num_chems, reactant, products)
      start_pos = end
  print("num new items = ", i+1)


In [2]:
start_item_num = 572
master_string = '''
! 1
ratj_t1(1,'EtOOH     ','PHOTON    ','MeCHO     ','HO2       ','OH        ',    &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',ti+t+st+r+cs,0,0,107),&
! 10
ratj_t1(10,'N2O5      ','PHOTON    ','NO3       ','NO2       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jn2o5     ',ti+s+t+st+r+cs,0,0,107),&
! 11
ratj_t1(11,'NO2       ','PHOTON    ','NO        ','O(3P)     ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.0,'jno2      ',ti+s+t+st+r+cs,0,0,107),&
! 17
ratj_t1(17,'PAN       ','PHOTON    ','MeCO3     ','NO2       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.0,'jpan      ',ti+t+st+r+cs,0,0,107) , &
! 18
! HONO not modelled in R
ratj_t1(18,'HONO      ','PHOTON    ','OH        ','NO        ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jhono     ',ti+t+st+cs,0,0,107) , &
! 20
ratj_t1(20,'Me2CO     ','PHOTON    ','MeCO3     ','MeOO      ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jaceto    ',t+ti+st+r+cs,0,0,107) , &
! 22 missing in A
ratj_t1(22,'i-PrOOH   ','PHOTON    ','Me2CO     ','HO2       ','OH        ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',t+st+r+cs,0,0,107) ,  &
! 35 - Same as MeOOH in CRI
ratj_t1(35,'MeCO3H    ','PHOTON    ','MeOO      ','OH        ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 36
ratj_t1(36,'BrCl      ','PHOTON    ','Br        ','Cl        ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jbrcl     ',s+st+cs,0,0,107) ,    &
! 37
ratj_t1(37,'BrO       ','PHOTON    ','Br        ','O(3P)     ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jbro      ',s+st+cs,0,0,107) ,    &
! 38
ratj_t1(38,'BrONO2    ','PHOTON    ','Br        ','NO3       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jbrna     ',s+st+cs,0,0,107) ,    &
! 41
ratj_t1(41,'OClO      ','PHOTON    ','O(3P)     ','ClO       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'joclo     ',s+st+cs,0,0,107) ,    &
! 42
ratj_t1(42,'NO        ','PHOTON    ','N         ','O(3P)     ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jno       ',s+st+cs,0,0,107) ,    &
! 46
ratj_t1(46,'ClONO2    ','PHOTON    ','Cl        ','NO3       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jclna     ',s+st+cs,0,0,107) ,    &
! 47
ratj_t1(47,'ClONO2    ','PHOTON    ','ClO       ','NO2       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jclnb     ',s+st+cs,0,0,107) ,    &
! 48
ratj_t1(48,'HCl       ','PHOTON    ','H         ','Cl        ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jhcl      ',s+st+cs,0,0,107) ,    &
! 49
ratj_t1(49,'HOCl      ','PHOTON    ','OH        ','Cl        ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jhocl     ',s+st+cs,0,0,107) ,    &
! 50
ratj_t1(50,'Cl2O2     ','PHOTON    ','Cl        ','Cl        ','O2        ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jcl2o2    ',s+st+cs,0,0,107) ,    &
! 51
ratj_t1(51,'CFCl3     ','PHOTON    ','Cl        ','Cl        ','Cl        ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jcfcl3    ',s+st+cs,0,0,107) ,    &
! 52
ratj_t1(52,'CF2Cl2    ','PHOTON    ','Cl        ','Cl        ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jcfc2     ',s+st+cs,0,0,107) ,    &
! 53
ratj_t1(53,'MeBr      ','PHOTON    ','Br        ','H         ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmebr     ',s+st+cs,0,0,107) ,    &
! 57
ratj_t1(57,'CS2       ','PHOTON    ','COS       ','SO2       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jcs2      ',s+st+cs,a,0,107),     &
! 81
ratj_t1(81,'NOA       ','PHOTON    ','MeCO3     ','HCHO      ','NO2       ',   &
     '          ', 0.0,0.0,0.0,0.0, 7.582e-02 ,'jno2      ',cs,0,0,107),       &
! 82
ratj_t1(82,'UDCARB8   ','PHOTON    ','EtOO      ','HO2       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 1.280e+00 ,'jno2      ',cs,0,0,107),       &
! 83
ratj_t1(83,'UDCARB11  ','PHOTON    ','RN10O2    ','HO2       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 1.100e+00 ,'jno2      ',cs,0,0,107),       &
! 84
ratj_t1(84,'UDCARB14  ','PHOTON    ','RN13O2    ','HO2       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 1.100e+00 ,'jno2      ',cs,0,0,107),       &
! 85
ratj_t1(85,'TNCARB26  ','PHOTON    ','RTN26O2   ','HO2       ','          ',   &
     '          ', 0.0,0.0,0.0,0.0, 9.640e+02 ,'jhchoa    ',cs,0,0,107),       &
! 98
ratj_t1(98,'RN10OOH   ','PHOTON    ','EtCHO     ','HO2       ','OH        ',   &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 99
ratj_t1(99,'RN13OOH   ','PHOTON    ','MeCHO     ','EtOO      ','OH        ',   &
     '          ', 0.0,0.0,0.0,0.0,  39.800,'jmhp      ',cs,0,0,107),          &
! 100
ratj_t1(100,'RN13OOH   ','PHOTON    ','CARB11A   ','HO2       ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0,  60.200,'jmhp      ',cs,0,0,107),          &
! 101
ratj_t1(101,'RN16OOH   ','PHOTON    ','RN15AO2   ','OH        ','          ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 102
ratj_t1(102,'RN19OOH   ','PHOTON    ','RN18AO2   ','OH        ','          ',  &
     '          ', 0.0,0.0,0.0,0.0,  100.000,'jmhp      ',cs,0,0,107),         &
! 103
ratj_t1(103,'EtCO3H    ','PHOTON    ','EtOO      ','OH        ','          ',  &
     '          ', 0.0,0.0,0.0,0.0,  100.000,'jmhp      ',cs,0,0,107),         &
! 104
ratj_t1(104,'HOCH2CO3H ','PHOTON    ','HCHO      ','HO2       ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0,  100.000,'jmhp      ',cs,0,0,107),         &
! 105
ratj_t1(105,'RN8OOH    ','PHOTON    ','EtOO      ','OH        ','          ',  &
     '          ', 0.0,0.0,0.0,0.0,  100.000,'jmhp      ',cs,0,0,107),         &
! 106
ratj_t1(106,'RN11OOH   ','PHOTON    ','RN10O2    ','OH        ','          ',  &
     '          ', 0.0,0.0,0.0,0.0,  100.000,'jmhp      ',cs,0,0,107),         &
! 107
ratj_t1(107,'RN14OOH   ','PHOTON    ','RN13O2    ','OH        ','          ',  &
     '          ', 0.0,0.0,0.0,0.0,  100.000,'jmhp      ',cs,0,0,107),         &
! 108
ratj_t1(108,'RN17OOH   ','PHOTON    ','RN16O2    ','OH        ','          ',  &
     '          ', 0.0,0.0,0.0,0.0,  100.000,'jmhp      ',cs,0,0,107),         &
! 109
ratj_t1(109,'RU14OOH   ','PHOTON    ','UCARB12   ','HO2       ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0,  25.200,'jmhp      ',cs,0,0,107),          &
! 110
ratj_t1(110,'RU14OOH   ','PHOTON    ','UCARB10   ','HCHO      ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0,  74.800,'jmhp      ',cs,0,0,107),          &
! 111
ratj_t1(111,'RU12OOH   ','PHOTON    ','CARB6     ','HOCH2CHO  ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 112
ratj_t1(112,'RU10OOH   ','PHOTON    ','MeCO3     ','HOCH2CHO  ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 113
ratj_t1(113,'NRU14OOH  ','PHOTON    ','NUCARB12  ','HO2       ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 114
ratj_t1(114,'NRU12OOH  ','PHOTON    ','NOA       ','CO        ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 114
ratj_t1(114,'NRU12OOH  ','PHOTON    ','NOA       ','CARB3     ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,119),          &
! 115
ratj_t1(115,'HOC2H4OOH ','PHOTON    ','HCHO      ','HCHO      ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 116
ratj_t1(116,'RN9OOH    ','PHOTON    ','MeCHO     ','HCHO      ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 117
ratj_t1(117,'RN12OOH   ','PHOTON    ','MeCHO     ','MeCHO     ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 118
ratj_t1(118,'RN15OOH   ','PHOTON    ','EtCHO     ','MeCHO     ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 119
ratj_t1(119,'RN18OOH   ','PHOTON    ','EtCHO     ','EtCHO     ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 120
ratj_t1(120,'NRN6OOH   ','PHOTON    ','HCHO      ','HCHO      ','NO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 121
ratj_t1(121,'NRN9OOH   ','PHOTON    ','MeCHO     ','HCHO      ','NO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 122
ratj_t1(122,'NRN12OOH  ','PHOTON    ','MeCHO     ','MeCHO     ','NO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 123
ratj_t1(123,'RA13OOH   ','PHOTON    ','CARB3     ','UDCARB8   ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 124
ratj_t1(124,'RA16OOH   ','PHOTON    ','CARB3     ','UDCARB11  ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 125
ratj_t1(125,'RA19OOH   ','PHOTON    ','CARB6     ','UDCARB11  ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 126
ratj_t1(126,'RTN28OOH  ','PHOTON    ','TNCARB26  ','HO2       ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 127
ratj_t1(127,'NRTN28OOH ','PHOTON    ','TNCARB26  ','NO2       ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 128
ratj_t1(128,'RTN26OOH  ','PHOTON    ','RTN25O2   ','OH        ','          ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 129
ratj_t1(129,'RTN25OOH  ','PHOTON    ','RTN24O2   ','OH        ','          ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 130
ratj_t1(130,'RTN24OOH  ','PHOTON    ','RTN23O2   ','OH        ','          ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 131
ratj_t1(131,'RTN23OOH  ','PHOTON    ','Me2CO     ','RTN14O2   ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 132
ratj_t1(132,'RTN14OOH  ','PHOTON    ','TNCARB10  ','HCHO      ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 133
ratj_t1(133,'RTN10OOH  ','PHOTON    ','RN8O2     ','CO        ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 134
ratj_t1(134,'RTX28OOH  ','PHOTON    ','TXCARB24  ','HCHO      ','HO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 135
ratj_t1(135,'RTX24OOH  ','PHOTON    ','TXCARB22  ','HO2       ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 136
ratj_t1(136,'RTX22OOH  ','PHOTON    ','Me2CO     ','RN13O2    ','OH        ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 137
ratj_t1(137,'NRTX28OOH ','PHOTON    ','TXCARB24  ','HCHO      ','NO2       ',  &
     'OH        ', 0.0,0.0,0.0,0.0, 100.000,'jmhp      ',cs,0,0,107),          &
! 138
ratj_t1(138,'UDCARB8   ','PHOTON    ','ANHY      ','HO2       ','HO2       ',  &
     '          ', 0.0,0.0,0.0,0.0, 7.200e-01 ,'jno2      ',cs,0,0,107),       &
! 139
ratj_t1(139,'UDCARB11  ','PHOTON    ','ANHY      ','HO2       ','MeOO      ',  &
     '          ', 0.0,0.0,0.0,0.0, 9.000e-01 ,'jno2      ',cs,0,0,107),       &
! 140
ratj_t1(140,'UDCARB14  ','PHOTON    ','ANHY      ','HO2       ','EtOO      ',  &
     '          ', 0.0,0.0,0.0,0.0, 9.000e-01 ,'jno2      ',cs,0,0,107),       &
! 141
ratj_t1(141,'TNCARB12  ','PHOTON    ','RN9O2     ','HOCH2CO3  ','          ',  &
     '          ', 0.0,0.0,0.0,0.0, 5.497e-02 ,'jno2      ',cs,0,0,107),       &
! 142
ratj_t1(142,'TNCARB11  ','PHOTON    ','RTN10O2   ','CO        ','HO2       ',  &
     '          ', 0.0,0.0,0.0,0.0, 1.500e+00 ,'jno2      ',cs,0,0,107),       &
! 143
ratj_t1(143,'MeO2NO2   ','PHOTON    ','MeOO      ','NO2       ','          ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jpna67    ',cs,0,0,107) ,         &
! 144
ratj_t1(144,'MeO2NO2   ','PHOTON    ','HCHO      ','HO2       ','NO3       ',  &
     '          ', 0.0,0.0,0.0,0.0, 100.000,'jpna33    ',cs,0,0,107),          &
'''

write_from_master(master_string, write_stashmaster_A, start_item_num)

1|    1 |   50 |  572 |EtOOH -> MeCHO + HO2 + OH J-values  |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000000000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  573 |N2O5 -> NO3 + NO2 J-values          |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000000000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  574 |NO2 -> NO + O(3P) J-values          |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000000000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |  

In [ ]:
# Make lots of new STASH items. Paste into suite app/um/rose-app.conf, then run tidyStashIndices macro.
# Should look like:
'''
[namelist:umstash_streq(1)]
dom_name='DALLTH'
isec=50
item=505
package=''
tim_name='T1H'
use_name='UPB'
'''

import numpy as np

dom_name = 'DALLTH'
isec = 50
tim_name = 'T1H'
use_name = 'UPB'
first_item = 572
last_item = 646

def makeItems(start, end, dom, sec, tim, use):
 num_items = end + 1 - start
 all_items = np.linspace(start, end, num_items, dtype = int)
 for index in range(num_items):
   item_num = all_items[index]
   each_string = "[namelist:umstash_streq({n})]\ndom_name='{d}'\nisec={s}\nitem={i}\npackage=''\ntim_name='{t}'\nuse_name='{u}'\n"\
                          .format(n = index + 1, d = dom, s = sec, i = item_num, t = tim, u = use)
   print(each_string)

makeItems(first_item, last_item, dom_name, isec, tim_name, use_name) 

[namelist:umstash_streq(1)]
dom_name='DALLTH'
isec=50
item=572
package=''
tim_name='T1H'
use_name='UPB'

[namelist:umstash_streq(2)]
dom_name='DALLTH'
isec=50
item=573
package=''
tim_name='T1H'
use_name='UPB'

[namelist:umstash_streq(3)]
dom_name='DALLTH'
isec=50
item=574
package=''
tim_name='T1H'
use_name='UPB'

[namelist:umstash_streq(4)]
dom_name='DALLTH'
isec=50
item=575
package=''
tim_name='T1H'
use_name='UPB'

[namelist:umstash_streq(5)]
dom_name='DALLTH'
isec=50
item=576
package=''
tim_name='T1H'
use_name='UPB'

[namelist:umstash_streq(6)]
dom_name='DALLTH'
isec=50
item=577
package=''
tim_name='T1H'
use_name='UPB'

[namelist:umstash_streq(7)]
dom_name='DALLTH'
isec=50
item=578
package=''
tim_name='T1H'
use_name='UPB'

[namelist:umstash_streq(8)]
dom_name='DALLTH'
isec=50
item=579
package=''
tim_name='T1H'
use_name='UPB'

[namelist:umstash_streq(9)]
dom_name='DALLTH'
isec=50
item=580
package=''
tim_name='T1H'
use_name='UPB'

[namelist:umstash_streq(10)]
dom_name='DALLTH'
isec=50


In [ ]:
# Pick out the names of reactions from STASHmaster_A entries and put them into a list.
# May be useful as titles for plotting in Python.

# Find out fixed positions in the stash entries.
stash = """1|    1 |   50 |  559 |STRATOSPHERIC HOBr PHOTOL J-VALUES  |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#"""

item_length = len(stash) # 342
start = stash.index("S")  # 23
end = stash.index("|", start) # 59
# Each entry is 342 chars long. The title is chars 23 to 58.

stash = """1|    1 |   50 |  500 |ORGANIC NITRATE PHOTOLYSIS RATE COEF|
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  501 |CO PROD VIA HCHO PHOT RAD RATE COEFS|
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  502 |CO PROD VIA HCHO PHOT MOL RATE COEFS|
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  503 |CO PROD VIA MGLY PHOTOL RATE COEFS  |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  504 |CO PROD VIA MISC PHOTOL RATE COEFS  |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  505 |STRAT OH PRODUCTION RATE COEFS      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  506 |O2+PHOTON->O(3P)+O(3P) RATE COEFS   |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  507 |STRAT Cl2O2 + PHOTON RATE COEFS     |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  508 |STRAT NO3 + PHOTON RATE COEFS       |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  509 |O1D PRODUCING PHOTOLYSIS RATE COEFS |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110011111111 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  510 |RU12NO3 -> NITRATE PHOTOLYSIS J-RATE|
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  511 |COS PHOTOLYSIS RATE COEFFICIENTS    |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000000000110011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  512 |H2SO4 PHOTOLYSIS RATE COEFFICIENTS  |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000000000110011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  513 |SO3 PHOTOLYSIS RATE COEFFICIENTS    |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000000000110011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  514 |MeONO2 -> NITRATE PHOTOLYSIS J-RATE |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  515 |NALD -> NITRATE PHOT J-RATE         |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  516 |ISON -> NITRATE PHOT J-RATE         |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  517 |EtONO2 -> NITRATE PHOT J-RATE       |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  518 |RN10NO3 -> NITRATE PHOT J-RATE      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  519 |i-PrONO2 -> NITRATE PHOT J-RATE     |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  520 |RN13NO3 -> MeCHO + NITRATE J-RATE   |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  521 |RN13NO3 -> CARB11A + NITRATE J-RATE |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  522 |RN16NO3 -> NITRATE PHOT J-RATE      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  523 |RN19NO3 -> NITRATE PHOT J-RATE      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  524 |RA13NO3 -> NITRATE PHOT J-RATE      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  525 |RA16NO3 -> NITRATE PHOT J-RATE      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  526 |RA19NO3 -> NITRATE PHOT J-RATE      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  527 |RTX24NO3 -> NITRATE PHOT J-RATE     |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  528 |RU10NO3 -> NITRATE PHOT J-RATE      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  529 |RU14NO3 -> NITRATE PHOT J-RATE      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010010 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  530 |CARB3 -> CO + HO2 PHOT J-RATE       |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  531 |CARB6 -> CO PHOTOL J-RATE           |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  532 |CARB3 -> CO + H2 PHOT J-RATE        |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  533 |CARB3 -> CO + HCHO PHOT J-RATE      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  540 |MeCHO -> CO PHOT J-RATE             |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  541 |EtCHO -> CO PHOT J-RATE             |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  542 |HOCH2CHO -> CO PHOT J-RATE          |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  543 |UCARB12 -> CO + MeCO3 PHOT J-RATE   |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  544 |UCARB12 -> CO + CARB7 PHOT J-RATE   |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  545 |NUCARB12 -> CO + HUCARB9 PHOT J-RATE|
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  546 |TNCARB10 -> CO PHOT J-RATE          |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  547 |RTN10OOH -> CO PHOT J-RATE          |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  548 |DHPCARB9 -> CO PHOT J-RATE          |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  549 |HPUCARB12 -> CO + HUCARB9 J-RATE    |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  550 |HPUCARB12 -> CO + CARB7 PHOT J-RATE |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  551 |HUCARB9 -> CO PHOT J-RATE           |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  552 |DHPR12OOH -> CO PHOT J-RATE         |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  553 |DHCARB9 -> CO PHOTOLYSIS J-RATE     |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  554 |NUCARB12 -> CO + CARB7 PHOT J-RATE  |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  555 |NO3 LOSS BY PHOTOLYSIS J-VALUES     |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000000001 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  556 |PEROXIDE DHPCARB9 PHOT J-VALUES     |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000010000000001 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  557 |PHOTOLYSIS OF WATER J-VALUES        |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000000000000000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  558 |PHOTOLYSIS OF METHANE J-VALUES      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  559 |STRATOSPHERIC HOBr PHOTOL J-VALUES  |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  560 |STRATOSPHERIC HOCl PHOTOL J-VALUES  |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  561 |STRATOSPHERIC HONO2 PHOTOL J-VALUES |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  562 |STRATOSPHERIC HO2NO2 PHOTOL J-VALUES|
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  563 |STRATOSPHERIC H2O2 PHOTOL J-VALUES  |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  564 |STRATOSPHERIC MeOOH PHOTOL J-VALUES |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  565 |O2+PHOTON->O(3P)+O(3P) RATE COEFS   |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000110000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  566 |O2+PHOTON->O(1D)+O(3P) RATE COEFS   |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000000000000000 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  567 |O3 -> O(1D) PHOTOLYSIS RATE COEFS   |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110011111111 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  568 |N2O -> O(1D) PHOTOLYSIS RATE COEFS  |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110011111111 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  569 |MACR -> CO PHOTOLYSIS J-VALUES      |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  570 |MACROOH -> CO PHOTOLYSIS J-VALUES   |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
#
1|    1 |   50 |  571 |MeCHO -> CO + CH4 PHOT J-VALUES     |
2|    0 |    0 |   17 |    1 |    2 |   10 |   11 |    0 |    0 |    0 |    0 |
3| 000000000000000000110000010011 | 00000000000000000001 |    3 |
4|    1 |    0 | -99  -99  -99  -99  -99  -99  -99  -99  -99  -99 |
5|    0 |    0 |    0 |   65 |    0 |    0 |    0 |    0 |    0 |
# """

names = []
whole_length = len(stash)
while start < whole_length:
  name = stash[start:end]
  names.append(name)
  start += item_length + 1
  end += item_length + 1

print(names)

['ORGANIC NITRATE PHOTOLYSIS RATE COEF', 'CO PROD VIA HCHO PHOT RAD RATE COEFS', 'CO PROD VIA HCHO PHOT MOL RATE COEFS', 'CO PROD VIA MGLY PHOTOL RATE COEFS  ', 'CO PROD VIA MISC PHOTOL RATE COEFS  ', 'STRAT OH PRODUCTION RATE COEFS      ', 'O2+PHOTON->O(3P)+O(3P) RATE COEFS   ', 'STRAT Cl2O2 + PHOTON RATE COEFS     ', 'STRAT NO3 + PHOTON RATE COEFS       ', 'O1D PRODUCING PHOTOLYSIS RATE COEFS ', 'RU12NO3 -> NITRATE PHOTOLYSIS J-RATE', 'COS PHOTOLYSIS RATE COEFFICIENTS    ', 'H2SO4 PHOTOLYSIS RATE COEFFICIENTS  ', 'SO3 PHOTOLYSIS RATE COEFFICIENTS    ', 'MeONO2 -> NITRATE PHOTOLYSIS J-RATE ', 'NALD -> NITRATE PHOT J-RATE         ', 'ISON -> NITRATE PHOT J-RATE         ', 'EtONO2 -> NITRATE PHOT J-RATE       ', 'RN10NO3 -> NITRATE PHOT J-RATE      ', 'i-PrONO2 -> NITRATE PHOT J-RATE     ', 'RN13NO3 -> MeCHO + NITRATE J-RATE   ', 'RN13NO3 -> CARB11A + NITRATE J-RATE ', 'RN16NO3 -> NITRATE PHOT J-RATE      ', 'RN19NO3 -> NITRATE PHOT J-RATE      ', 'RA13NO3 -> NITRATE PHOT J-RATE      ',

In [ ]:
# Rewrite asad_flux_dat to be more readable.

big_string = """
TYPE(asad_flux_defn), PARAMETER :: asad_trop_ox_budget_prod(26) =              &
! Production of Ox
       [ asad_flux_defn('RXN',50001,'B',.TRUE.,0,4,                            &
       ['HO2       ','NO        '],                                            &
       ['OH        ','NO2       ','          ','          ']),                 &
       asad_flux_defn('RXN',50002,'B',.TRUE.,0,5,                              &
       ['MeOO      ','NO        '],                                            &
       ['HO2       ','HCHO      ','NO2       ','          ']),                 &

! NO + RO2 reactions: sum into STASH section 50 item 3
       asad_flux_defn('RXN',50003,'B',.TRUE.,0,5,                              &
       ['EtOO      ','NO        '],                                            &
       ['MeCHO     ','HO2       ','NO2       ','          ']),                 &
       asad_flux_defn('RXN',50003,'B',.TRUE.,0,5,                              &
       ['MeCO3     ','NO        '],                                            &
       ['MeOO      ','CO2       ','NO2       ','          '])
"""

i, pos = 0, 0
while pos > -1:
  pos = big_string.find('asad_flux_defn(', i)
  i = pos + 1
  print(pos)

110
351
650
893
-1
